# Fine Tuning Llama 2 with Uncleaned Data

This is one of four approaches that we took with Llama model fine-tuning. evaluation are cross the approaches for choose the best approach.

# Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Data Folder Directry
main_dir = '/content/drive/MyDrive/Colab Notebooks/Data/'

In [ ]:
import os
import pandas as pd

In [ ]:
sub_dir = 'Ubuntu_Dialogue_Corpus/Ubuntu-dialogue-corpus/dialogueText_301.csv'
full_file_path = os.path.join(main_dir, sub_dir)

df_ubuntu = pd.read_csv(full_file_path)
df_ubuntu

,folder,dialogueID,date,from,to,text
0,301,1.tsv,2004-11-23T11:49:00.000Z,stuNNed,NaN,any ideas why java plugin takes so long to load?
1,301,1.tsv,2004-11-23T11:49:00.000Z,crimsun,stuNNed,java 1.4?
2,301,1.tsv,2004-11-23T11:49:00.000Z,stuNNed,crimsun,yes
3,301,1.tsv,2004-11-23T11:49:00.000Z,crimsun,stuNNed,java 1.5 loads _much_ faster
4,301,1.tsv,2004-11-23T11:50:00.000Z,stuNNed,crimsun,noneus: how can i get 1.5 is there a .deb some...
...,...,...,...,...,...,...
16587825,32,1783.tsv,2007-11-15T03:38:00.000Z,koyo001,NaN,thanks
16587826,32,1783.tsv,2007-11-15T03:39:00.000Z,koyo001,NaN,does anyone know something
16587827,32,1783.tsv,2007-11-15T03:39:00.000Z,neverblue,NaN,"no, no one knows everything"
16587828,32,1783.tsv,2007-11-15T03:40:00.000Z,koyo001,ikonia,the camera doesnt work


# Simple Processing Combine Data with same dialogueID

In [ ]:
# Group the 'df_ubuntu' DataFrame by the 'dialogueID' column.
grouped = df_ubuntu.groupby('dialogueID')
# Create an empty list to store pairs of dialogues.
dialogue_pairs = []

# Loop over each group from the grouped DataFrame.
for _, group in grouped:
    # Convert the 'text' column of each group to a list of messages.
    messages = group['text'].tolist()
    # Loop through each message in the list except the last one.
    for i in range(len(messages) - 1):
        # Append a dictionary to 'dialogue_pairs' list
        dialogue_pairs.append({
            'input': str(messages[i]),
            'output': str(messages[i + 1])
        })

In [ ]:
# Split Train and eval dataset 80/20
train_size = int(0.8 * len(dialogue_pairs))
train_pairs = dialogue_pairs[:train_size]
eval_pairs = dialogue_pairs[train_size:]

In [ ]:
import json
save_path = '/content/drive/MyDrive/Colab Notebooks/Data/'
# Write training pairs to `ubuntu_train.jsonl` in the specified directory
with open(save_path + 'ubuntu_train.jsonl', 'w') as file:
    for pair in train_pairs:
        file.write(json.dumps(pair) + '\n')

# Write evaluation pairs to `ubuntu_eval.jsonl` in the specified directory
with open(save_path + 'ubuntu_eval.jsonl', 'w') as file:
    for pair in eval_pairs:
        file.write(json.dumps(pair) + '\n')


In [ ]:
import json
save_path = '/content/drive/MyDrive/Colab Notebooks/Data/'

In [ ]:
!pip install -q huggingface_hub
!pip install -q -U trl transformers accelerate peft
!pip install -q -U datasets bitsandbytes einops wandb
!pip install  -q ipywidgets
!pip install -q scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
from datasets import load_dataset
# Load Training and eval datasets
train_dataset = load_dataset('json', data_files=save_path + 'ubuntu_train.jsonl', split='train')
eval_dataset = load_dataset('json', data_files=save_path + 'ubuntu_eval.jsonl', split='train')



Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

# Llama model fine tuning code learned from Carroll, H. (2023, Oct 13). Fine-Tuning Llama 2 7B on your own data (5134ba9) GitHub Jupyter Notebook https://github.com/brevdev/notebooks/blob/main/llama2-finetune.ipynb

In [ ]:
def formatting_func(example):
    text = f"### Context: {example['input']}\n ### Reply: {example['output']}"
    return [text]

In [ ]:
# Fine tuning code from https://github.com/brevdev/notebooks/blob/main/llama2-finetune.ipynb
# Define base model name
base_model_name = "meta-llama/Llama-2-7b-hf"
# Define the configuration for 'BitsAndBytes' quantization method.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, #Indicates to load the model using 4-bit precision.
    bnb_4bit_quant_type="nf4", #Specifies the type of 4-bit quantization, here it's set to "nf4".
    bnb_4bit_compute_dtype=torch.float16, # Sets the data type for computations to float16.
)
# Load a pre-trained causal language model.
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name, # Model name to load.
    quantization_config=bnb_config, #Apply the previously defined quantization config.
    device_map="auto", # Automatically map the model to available devices
    trust_remote_code=True, #Trust any custom code from the remote model source.
    use_auth_token=True # Use an authentication token for accessing private models or repositories.
)
# Disable caching for the loaded model to save memory.
base_model.config.use_cache = False
# Set the `pretraining_tp` attribute of the model's configuration to 1.
base_model.config.pretraining_tp = 1
# Load the tokenizer corresponding to the `base_model_name`.
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
# Set the padding token of the tokenizer to be its end-of-sequence token.
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:479: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [ ]:
# Fine tuning code from https://github.com/brevdev/notebooks/blob/main/llama2-finetune.ipynb
output_dir = "./Llama-2-7b-hf-Unclean-Data"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=50,
    max_steps=1000,
    logging_dir="./logs",        # Directory for storing logs
    save_strategy="steps",       # Save the model checkpoint every logging step
    save_steps=50,                # Save checkpoints every 50 steps
    evaluation_strategy="steps", # Evaluate the model every logging step
    eval_steps=50,               # Evaluate and save checkpoints every 50 steps
    do_eval=True                 # Perform evaluation at the end of training
)

In [ ]:
# Fine tuning code from https://github.com/brevdev/notebooks/blob/main/llama2-finetune.ipynb
# Define the configuration for the 'LoRA' (Layer-wise Relevance Analysis)
peft_config = LoraConfig(
    lora_alpha=16, # Indicates the dimension of LoRA adaptation. Set to 16.
    lora_dropout=0.1, #  Specifies the dropout rate for LoRA. Set to 0.1.
    r=64, # The rank for the projection matrix in LoRA. Set to 64.
    bias="none", # Specifies the type of bias for LoRA. Here, no bias is used.
    task_type="CAUSAL_LM", # Sets the type of task for which LoRA is configured to Causal Language Model
)

In [ ]:
# Fine tuning code from https://github.com/brevdev/notebooks/blob/main/llama2-finetune.ipynb
# Set the maximum sequence length for tokenization and processing
max_seq_length = 512
# Initialize the SFTTrainer, which will handle training and evaluation of the model
trainer = SFTTrainer(
    model=base_model, #The base model to be trained.
    train_dataset=train_dataset, # The dataset to be used for training.
    eval_dataset=eval_dataset, # The dataset to be used for evaluation.
    peft_config=peft_config, # Configuration for the 'LoRA' (Layer-wise Relevance Analysis) method
    formatting_func=formatting_func, # A function that formats the dataset to be suitable for training
    max_seq_length=max_seq_length, # The maximum number of tokens in a sequence.
    tokenizer=tokenizer, # The tokenizer
    args=training_args, # Additional arguments specifying training parameters
)

# Start the training process
trainer.train()

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/12993377 [00:00<?, ? examples/s]

Map:   0%|          | 0/3248345 [00:00<?, ? examples/s]

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
50,2.684200,2.625473
100,2.609400,2.600198
150,2.598300,2.585887
200,2.592100,2.577899
250,2.573100,2.569283
300,2.563000,2.564968
350,2.571100,2.561513
400,2.565300,2.558302
450,2.569000,2.555994
500,2.555600,2.552957


TrainOutput(global_step=1000, training_loss=2.564831787109375, metrics={'train_runtime': 45992.8971, 'train_samples_per_second': 0.348, 'train_steps_per_second': 0.022, 'total_flos': 1.6723820534562816e+17, 'train_loss': 2.564831787109375, 'epoch': 1.23})

In [ ]:
import os
# Store LoRA Adapter
output_dir1 = "/content/drive/MyDrive/Colab Notebooks/Data/"
output_dir1 = os.path.join(output_dir1, "final_checkpoint")
trainer.model.save_pretrained(output_dir1)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from peft import PeftModel
import os
# load LoRA Adapter
output_dir1 = "/content/drive/MyDrive/Colab Notebooks/Data/"
output_dir1 = os.path.join(output_dir1, "final_checkpoint")

In [ ]:
base_model_name="meta-llama/Llama-2-7b-hf"
# Define the configuration for 'BitsAndBytes' quantization method.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
# Load a pre-trained causal language model.
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [ ]:
# Setup tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# joint base model with LoRA adapter
model = PeftModel.from_pretrained(base_model, output_dir1)


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer


In [ ]:
# Load Eval dataset
eval_dataset = load_dataset('json', data_files='/content/drive/MyDrive/Colab Notebooks/Data/ubuntu_eval.jsonl', split='train')


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

# BLEU and ROUGE SCORES

In [ ]:
!pip install -q rouge_score

  Preparing metadata (setup.py) ... done


In [ ]:
from datasets import load_metric

model.to("cuda")
model.eval()

responses = []

# Convert the first 1000 examples into a list of dictionaries
subset_eval = eval_dataset.select(range(1000)).with_format("python")

for example in subset_eval:
    # Check if example is a dictionary
    if not isinstance(example, dict):
        print(f"Unexpected data type within dataset: {type(example)}")
        continue  # Skip this iteration if it's not a dictionary

    # Ensure 'input' and 'output' keys exist
    if 'input' not in example or 'output' not in example:
        print(f"Missing keys in example: {example.keys()}")
        continue  # Skip this iteration if keys are missing

    eval_prompt = f"### Context: {example['input']}\n ### Reply:"

    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        generated_response = tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True)
        responses.append(generated_response)

# Extract the actual responses (ground truths) for the first 100 examples
ground_truths = [example['output'] for example in subset_eval if 'output' in example]

# Tokenize the responses and ground truths using the Transformers tokenizer
tokenized_responses = [tokenizer.tokenize(response) for response in responses]
tokenized_ground_truths = [[tokenizer.tokenize(gt)] for gt in ground_truths]

# Compute BLEU and ROUGE metrics
bleu_metric = load_metric('bleu')
rouge_metric = load_metric('rouge')

bleu_score = bleu_metric.compute(predictions=tokenized_responses, references=tokenized_ground_truths)
rouge_score = rouge_metric.compute(predictions=responses, references=ground_truths)

print("BLEU Score:", bleu_score)
print("ROUGE Score:", rouge_score)

<ipython-input-13-c7ad26943c9a>:38: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric('bleu')


BLEU Score: {'bleu': 0.005802971644633009, 'precisions': [0.030994651002526828, 0.006361049531813519, 0.0030527474727254527, 0.0018840628469535376], 'brevity_penalty': 1.0, 'length_ratio': 8.62036775106082, 'translation_length': 121892, 'reference_length': 14140}
ROUGE Score: {'rouge1': AggregateScore(low=Score(precision=0.03280758284698293, recall=0.22601563318371704, fmeasure=0.05370407724303023), mid=Score(precision=0.03523516820168231, recall=0.24109979924244446, fmeasure=0.057549430741402965), high=Score(precision=0.03793993208382937, recall=0.25489605556535866, fmeasure=0.06143773176052148)), 'rouge2': AggregateScore(low=Score(precision=0.004228911240177544, recall=0.03266002395740293, fmeasure=0.007072247828530782), mid=Score(precision=0.005297562530594871, recall=0.04048666131084151, fmeasure=0.008848459792582641), high=Score(precision=0.0065424085151723705, recall=0.048392813010202027, fmeasure=0.010852825468164871)), 'rougeL': AggregateScore(low=Score(precision=0.026843195466

In [ ]:
print(responses)

["### Context: yes\n ### Reply:  that's the whole point of the program, it's a live CD that you can boot up from a CD and install to your hard drive.  I have done that myself, but it's a lot of work.  I'm not sure if there is a way to install the program to a USB stick, but I don't think there is.  The program is designed to be used on a CD, and then you install the rest of the system from the"]


In [ ]:
print(example)

{'input': 'yes', 'output': 'looking now...'}


In [ ]:
from datasets import load_metric

In [ ]:
ground_truths = [example['output'] for example in eval_dataset]


In [ ]:
# Load the metrics
bleu_metric = load_metric('bleu')
rouge_metric = load_metric('rouge')

# Compute BLEU
bleu_score = bleu_metric.compute(predictions=responses, references=[[gt] for gt in ground_truths])

# Compute ROUGE
rouge_score = rouge_metric.compute(predictions=responses, references=ground_truths)

print("BLEU Score:", bleu_score)
print("ROUGE Score:", rouge_score)


In [ ]:
eval_prompt = "What is the command to install a package in Ubuntu?"

# Tokenize the prompt
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

# Set the model to evaluation mode
model.eval()

# Generate a response with a strict token limit
with torch.no_grad():
    generated_response = model.generate(**model_input, max_new_tokens=100)[0]
    decoded_response = tokenizer.decode(generated_response, skip_special_tokens=True)

# Print the provided question and the model's generated response
#print("Question:", eval_prompt)
print("Answer:", decoded_response.replace(eval_prompt, "").strip())





Answer: I've been trying to install a package in Ubuntu, and I have been using `sudo apt-get install package_name`, but it doesn't seem to be working. I've tried `sudo apt-get install package_name` and `sudo apt-get install package_name` and both have failed.

Answer: `sudo apt-get install package_name`

Comment: This is not correct, you can use the -y flag to


In [ ]:
print(decoded_response)

What is the command to install a package in Ubuntu?

I've been trying to install a package in Ubuntu, and I have been using `sudo apt-get install package_name`, but it doesn't seem to be working. I've tried `sudo apt-get install package_name` and `sudo apt-get install package_name` and both have failed.

Answer: `sudo apt-get install package_name`

Comment: This is not correct, you can use the -y flag to


In [ ]:
import re

eval_prompt = "Context: does ubuntu come with a firewall by default?\n Reply: "

# Tokenize the prompt
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

# Set the model to evaluation mode
model.eval()

# Generate a response with a strict token limit
with torch.no_grad():
    generated_response = model.generate(**model_input, max_new_tokens=100)[0]
    decoded_response = tokenizer.decode(generated_response, skip_special_tokens=True)

def extract_answer(decoded_response):
    match = re.search(r'(Reply):\s*(.*)', decoded_response)
    if match:
        return match.group(2)
    else:
        return decoded_response

# Print the provided question and the model's generated response
print("Question:", eval_prompt)
print()
print("Answer:", extract_answer(decoded_response))


Question: Context: does ubuntu come with a firewall by default?
 Reply: 

Answer: nope, you'd need to install one.  there are a few to choose from.


In [ ]:
print(decoded_response)

Context: does ubuntu come with a firewall by default?
 Reply:  nope, you'd need to install one.  there are a few to choose from.
 Reply:  thanks, I'll look into that.
 Reply:  i have a problem with the terminal, when i type something like ls or cat it gives me the output but when i type something like ls -l it does not. any idea what could be the problem?
 Reply:  what's the output?
 Reply:  i have a


# Reference:

Carroll, H. (2023, Oct 13). Fine-Tuning Llama 2 7B on your own data (5134ba9). Brev.dev/notebooks. https://github.com/brevdev/notebooks/blob/main/llama2-finetune.ipynb